**IOC Case Study: MAC Flooding (Layer 2 Switch Table Overload)**

1. Attacker Analogy 
Analogy:
Imagine a hotel concierge desk that tracks which guest is in which room so that mail and room service go to the correct people. An attacker walks into the lobby and gives the concierge a list of thousands of fake guest names with made-up room numbers. The concierge runs out of space to keep track and gives up. Now, instead of sending mail to the right room, they just broadcast every letter and meal tray to every room on every floor. The attacker—who’s just booked a random room—now gets copies of everyone's deliveries and can quietly observe or tamper with them.

Narrative Integration:

1. The precondition for MAC flooding  (also known as switch flooding or CAM table flooding)
The attacker must already be connected to the same Layer 2 network segment as the target traffic. This could occur in one of two ways:
A compromised host is already under the attacker's control. That host's network interface card (NIC) is used to transmit the flood of fake MAC addresses.
Or the attacker is physically plugged in or wirelessly joined to the LAN (e.g., rogue device on poorly secured guest Wi-Fi).

Either way, the attacker has Layer 2 access, and the flood comes from a NIC already accepted by the switch.
This is MAC flooding: the attacker poisons the switch's internal memory by overwhelming it with fake MAC address mappings. As the switch fails open into broadcast mode, the attacker’s NIC (network interface card) now receives traffic not meant for it. This is the equivalent of passive surveillance, and it's a powerful precursor to deeper lateral movement or credential theft—especially in flat or poorly segmented networks.

The attacker’s NIC = the NIC of the compromised host under the attacker’s control.
It must be connected to the same Layer 2 network as the target traffic.
That NIC floods the switch with MACs → switch fails open → traffic gets mirrored to the attacker's port → attacker sniffs it using the NIC.

**2. Source of IOC (Telemetry Origin)**
Packet Capture Sensor + EDR + NetFlow Exporters
In our battlefield model, this indicator originates from a combination of sensors:

Packet Capture Sensor: Abnormal flood of Ethernet frames with changing source MACs.

EDR Telemetry: Unusual process spawning packet generation tools (e.g., macof) on an endpoint.

NetFlow Logs: Significant spike in Layer 2 broadcast traffic patterns from a single host.

**3. Triage Framework Declaration**
Primary Triage: Host System Activity Protocol – attacker uses a compromised NIC to flood the local switch.
Secondary Triage: Network Triage Protocol (pivot) – required to confirm downstream Layer 2 anomalies (e.g., CAM table flooding, broadcast storms, traffic mirroring).
This makes it clear:
Source = host
Symptom = network
Triager’s job = observe both

**4. Investigative Tools (Host Triage Protocol)**
Host Triage Protocol – Softened Interpretation Version
When you're dealing with MAC flooding from a host, you're not going to see red flags waving in the system logs that scream, "The attacker is spoofing the switch!" Layer 2 attacks are quiet by nature. But if you're smart about where you look, the host tells you a story.

First, the Windows Event Logs. If the attacker launches a MAC flood tool like macof, that’s a process execution. Windows logs it—as long as you're watching for it. Tools like cmd.exe, python.exe, or powershell.exe running suspicious scripts can all show up under Event ID 4688. And if the tool installs itself as a background service, Event ID 7045 is your friend.

Second, EDR telemetry is your magnifying glass. Unlike basic logs, EDR tools can show you who launched what, from where, and what happened next. If a simple script opened a raw socket and started talking to the NIC in odd ways, good EDR will notice. That’s especially useful when the activity isn’t malware—it’s just weird.

Third, dig into the file system and registry. The attacker needs to store or stage that flooding tool somewhere. Common places: temporary folders, startup keys, or renamed drivers hidden in System32. You’re looking for things like unsigned packet drivers, autoruns, or scripts dropped where they shouldn’t be.

Last, if the trail’s gone cold, memory doesn’t lie. Use memory capture tools to check for flood tools running in RAM. Look for open NIC sockets, injected drivers, or tools that were never written to disk at all. Memory tells the story of what’s active—not just what was left behind.

**5. OS Layer with Key Behaviors**

OS Layer	Observed or Likely Attacker Behavior
Layer 1: Process Execution Environment	Attacker launches a MAC flooding utility (e.g., macof.exe, custom Python script) to begin overwhelming the switch with forged MAC addresses.
Layer 2: Startup and Persistence Infrastructure	Tool may be configured to auto-run via registry autorun key, scheduled task, or service to ensure the attack continues across reboots.
Layer 3: Background Service and Execution Context	Flooding tool runs silently as a long-running background task or hidden service—no visible user interface, often suppressed in Task Manager.
Layer 5: Monitoring and Detection Evasion	Attacker may disable or suppress Windows Event Logging, tamper with EDR, or rename tools to avoid detection by string-based scanners.
Layer 6: Network Communication Context	NIC is placed under attacker control; forged frames are sent at high volume from a single interface. This results in CAM table exhaustion and traffic broadcast behavior across the LAN segment.

Softened Interpretation Version
From the operating system’s point of view, this attack touches multiple layers of host behavior. It’s not just about sending packets—it’s about how the attacker lives inside the system.

It begins at Layer 1: the attacker runs a tool—maybe macof.exe, maybe a Python script—to start flooding the switch. That’s the first real move: process execution.

At Layer 2, the attacker doesn’t want to run this manually every time. They likely set it to auto-start with the system using the registry or a scheduled task. That way, the flooding is persistent—even after a reboot.

Then Layer 3 comes in: the tool runs quietly in the background. There’s no visible interface. It’s just there, doing its job—overwhelming the switch without drawing attention.

By Layer 5, the attacker starts thinking about stealth. They might rename the tool or disable system logging. They may suppress EDR alerts or clear logs after tool execution.

And finally, Layer 6: the NIC becomes their weapon. From that one compromised interface, the attacker starts sending out fake MAC addresses as fast as the system allows. The switch gets overwhelmed. The network gets noisy. And traffic meant for others starts flowing through the attacker’s host.

 When Layer 4 Is Relevant in This Case
You’re not just studying MAC flooding. You're studying an attacker who launched MAC flooding from within a compromised host. Therefore:
You should investigate how they got onto the system:
Local login? RDP?
Stolen credentials?
Pass-the-hash?
Guest Wi-Fi exploit?
You should also check if login traces (e.g., Event ID 4624, 4625, 4648) help identify:
The attacker’s entry point
Their privilege level
Whether the host is part of lateral movement from another asset


**6. Cross-Layer Interaction Pivots**
Layer 2 → Layer 6: Persistence or scheduled task leads to launch of broadcast flood on Layer 6.
Layer 6 → Layer 3: If IP traffic is captured post-flooding, attacker may escalate to IP spoofing or credential theft via Layer 3 analysis.
Layer 6 → Layer 1 (External): Hardware NIC is being manipulated to send malformed or randomized MAC source traffic.

**7. OSI Layer Relevance**
OSI Layer Relevance 
OSI Layer	Role in Attack
Layer 2 – Data Link	Primary target. Attacker floods the switch with forged MAC addresses, filling the CAM table. Once full, the switch fails open and broadcasts all incoming frames to all ports, mimicking a hub. This enables passive interception from any host on the same segment.
Layer 3 – Network	Collateral impact. Broadcast behavior can lead to abnormal IP routing behavior or misdelivery of IP-based traffic. In flat networks, Layer 3 segmentation may be compromised, and attackers may see Layer 3 traffic not destined for them.
Layer 4 – Transport (Optional/Contextual)	If the attacker uses a tool that filters or captures TCP sessions (e.g., credential harvesting from unencrypted sessions), transport-layer data becomes accessible.
Layer 7 – Application (Optional)	If successful, the attacker may intercept application-layer data (e.g., HTTP requests, email, SMB), especially if encryption is absent or weak. This is often the goal of the flooding: to gain access to usable plaintext content.

Softened Interpretation Version
The MAC flooding attack sits squarely in OSI Layer 2—the data link layer. That’s where switches live. That’s where MAC addresses are managed. But like most attacks, it doesn’t stay in one layer.

The attacker doesn’t open a console and “manipulate Layer 2” in abstract terms. They launch a tool—at the application layer—that sends thousands of fake MAC frames. That’s an action from Layer 7 downward. The tool sends these packets using Layer 4 sockets, which are handed to the NIC at Layer 2.

Once the switch is overwhelmed, its behavior changes: it starts acting like a hub, broadcasting traffic everywhere. That’s Layer 2 failing open. But the attacker doesn’t care about Layer 2 itself—they care about what’s inside the packets: the IP addresses (Layer 3), the TCP sessions (Layer 4), and the credentials or data (Layer 7).

So while the switch is the technical target, the attacker’s intent is layered. The flood is a doorway—not the goal.

**8. Attacker Behavior Interpretation**

This attacker doesn’t kick down the door. They slide in—quietly. They already have access, maybe by compromising a workstation, maybe by plugging into a forgotten wall jack. They’re not noisy. They’re patient.

Once inside, they don’t ask for passwords or try to exploit the system immediately. Instead, they watch. But watching takes positioning. So they flood the network switch with lies—thousands of fake MAC addresses. The switch, overwhelmed, gives up trying to sort traffic by destination. It just broadcasts everything to everyone.

That’s what the attacker wants. They want the quiet background noise of the network to suddenly start flowing through them. They want to see what users are logging into. Who’s connecting to what. Where the traffic goes at 9:00 AM versus midnight.

They may not stop here. Once they’ve got the broadcast behavior working, they might add ARP spoofing to redirect specific flows. Or spin up a fake DHCP server to tell devices to route traffic through them. It’s all modular. It all starts with the same quiet trick: overwhelming the switch so it trusts no one and gives everything away.

**9. Defender Action Summary**
Section 9 – Defender Action Summary (Technical Version)
A layered response is required to contain and analyze MAC flooding attacks. Steps should be taken both at the network switch level and within the compromised host system.

A. Network-Level Actions (Switch / Broadcast Domain)
Isolate the Affected Switch Port Immediately

Physically disconnect or administratively shut down the port observed broadcasting excessive MAC activity.

Use switch logs or broadcast storm control alerts to identify high-traffic offenders.

Inspect Switch CAM Table Behavior

Use commands such as show mac address-table (Cisco) or equivalent to identify overflow or instability.

Check for rapid MAC churn (many MACs learned on a single port over a short time).

Review Network Telemetry

NetFlow or sFlow may show excessive internal broadcast traffic or unusual port-pair flows.

Look for symptoms of broadcast storms or duplicated frame delivery.

Enforce Port-Level Security

Implement port security policies: limit number of allowed MAC addresses per port (e.g., switchport port-security maximum 2)

Enable dynamic ARP inspection and DHCP snooping where available.

Consider MAC address sticky binding for known safe hosts.

B. Host-Level Actions (Triage and Forensics)
Identify and Investigate the Flooding Host

Cross-reference switch logs with DHCP leases, ARP tables, and inventory to identify host by MAC/IP.

Correlate with EDR to identify the endpoint responsible for initiating the flood.

Examine Host for Credential Activity

Review Windows Event IDs 4624 (Logon Success), 4625 (Logon Failures), and 4648 (Explicit Credential Use).

Determine whether attacker authenticated via compromised credentials—this informs lateral movement risk.

Check for Execution of Flooding Tools

Analyze Event ID 4688 (Process Creation) for execution of tools like macof, ettercap, or custom scripts.

Examine parent-child process relationships, binary locations, and signature status.

Search for Persistence or Re-Execution Mechanisms

Check registry and scheduled tasks for autorun configuration of MAC flooding utilities.

Perform Memory Forensics if Necessary

Dump volatile memory and analyze with tools like Volatility to detect active NIC usage, raw sockets, or memory-resident packet tools.

C. Containment and Recovery
Quarantine compromised host pending further investigation

Invalidate any exposed credentials if traffic interception was successful

Audit remaining network segments for signs of similar flooding or port abuse

Review NAC policy coverage and switch security posture

Section 9 – Defender Action Summary (Softened Interpretation Version)
As a defender, you don’t just respond to the flood—you have to trace its source, contain the damage, and patch the holes.

Start at the switch. If one port is flooding the entire network, shut it down. Look at the MAC address table. Is one port claiming to be a hundred different devices? That’s your offender. Is the switch log warning about MAC churn or broadcast storms? That’s your signal.

Then trace the cable. Which host is plugged into that port? That’s where your host triage begins.

On the host, start asking questions:

Who logged in?

What processes ran?

Did macof.exe come from cmd.exe or a PowerShell script?

Is there a registry key set to re-launch this tool at reboot?

And don’t forget memory—just because the tool’s not on disk doesn’t mean it’s gone. Memory forensics can catch it mid-act.

Once you’ve confirmed the threat, quarantine the device. Reset any credentials that may have been exposed in the broadcast traffic. And don’t stop at cleanup—lock down your switches. Limit MACs per port. Enable port security. If this happened once, it can happen again—unless your defenses learn faster than the attacker.



**10. Attacker Strategy Notes**
Attacker Profile and Timing
The attacker executing a MAC flooding operation is already inside the network perimeter—likely post-compromise. Their mindset is quiet positioning: gain visibility before making noise. This tactic is often deployed immediately after initial access, during the lateral movement prep phase, when they are probing the network and seeking paths to privilege escalation or data access.

The goal isn’t disruption—it’s observation. MAC flooding allows the attacker to mirror internal network traffic to their NIC, helping them learn who’s talking to whom, and how.

They know the attack is stealthy at first—until the CAM table overloads and the switch broadcasts packets indiscriminately. That’s the threshold they manage carefully. A skilled attacker may rate-limit their flood, staying just below alert thresholds.

Tools and Tactics
macof – from the DSniff suite; rapidly generates fake MAC/IP pairs to exhaust switch memory.

Custom SCAPI-based tools – allow raw Ethernet frame generation for controlled flooding.

Python or PowerShell wrappers – sometimes used to launch and hide the flooding tool within larger reconnaissance scripts.

Driver abuse or raw socket manipulation – low-level NIC control for packet crafting.

Strategic Objectives
Capture authentication flows – especially if plaintext credentials, NTLM handshakes, or Kerberos requests are present in broadcast.

Fingerprint internal protocols – identify use of SMB (Server Message Block), LDAP, or HTTP inside the segment.

Setup for follow-on attacks such as:

SMB Hijacking
The attacker intercepts legitimate SMB traffic intended for a file server. If they can respond faster or more convincingly, they can impersonate the file server and receive session requests, allowing them to access file shares or redirect content.

NTLM Relay
A classic credential abuse tactic: the attacker captures an NTLM authentication attempt (e.g., from SMB or HTTP), then relays it to another internal system in real-time—without cracking the hash. This enables pass-through authentication, letting them impersonate the user on a second target.

Psychological Framing
This attacker is a ghost inside the LAN, operating below the noise threshold. They're not breaking down doors—they’re walking the halls quietly, listening at every room. MAC flooding is their way of making the building speak: a method of forcing the network to expose what it’s doing, so the attacker can decide what to do next.

They act with timing and patience. The goal is not damage—it’s positioning. And when the right moment arrives, they’re ready to pivot into something louder: spoofing, relaying, hijacking, or privilege escalation. But it all starts here—with a quiet flood, and a NIC that catches everything the network no longer knows how to deliver.

